<a href="https://colab.research.google.com/github/ommusuc/lecture-ai-engineering/blob/master/ex03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment for

In [ ]:
%%bash
wget --no-check-certificate -O "chap4lib.7z" "https://docs.google.com/uc?export=download&id=19oqo3xnLU8uim52ponISKpWnrmMAZGVQ"
7z x chap4lib.7z


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
1 file, 1458 bytes (2 KiB)

Extracting archive: chap4lib.7z
--
Path = chap4lib.7z
Type = 7z
Physical Size = 1458
Headers Size = 130
Method = LZMA2:6k
Solid = -
Blocks = 1


Would you like to replace the existing file:
  Path:     ./chap4lib.py
  Size:     5251 bytes (6 KiB)
  Modified: 2025-05-25 16:46:08
with the file from archive:
  Path:     chap4lib.py
  Size:     5251 bytes (6 KiB)
  Modified: 2025-05-25 16:46:08
? (Y)es / (N)o / (A)lways / (S)kip all / A(u)to rename all / (Q)uit? 

--2025-06-04 06:31:02--  https://docs.google.com/uc?export=download&id=19oqo3xnLU8uim52ponISKpWnrmMAZGVQ
Resolving docs.google.com (docs.google.com)... 173.194.206.139, 173.194.206.138, 173.194.206.113, ...
Connecting to docs.google.com (docs.google.com)|173.194.206.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=19oqo3xnLU8uim52ponISKpWnrmMAZGVQ&export=download [following]
--2025-06-04 06:31:02--  https://drive.usercontent.google.com/download?id=19oqo3xnLU8uim52ponISKpWnrmMAZGVQ&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 173.194.193.132, 2607:f8b0:4001:c0f::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|173.194.193.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1458 (1.4K) [application/octet-stream]
Saving to: ‘chap4lib.7z’

     0K .                                                     100% 14

CalledProcessError: Command 'b'wget --no-check-certificate -O "chap4lib.7z" "https://docs.google.com/uc?export=download&id=19oqo3xnLU8uim52ponISKpWnrmMAZGVQ"\n7z x chap4lib.7z\n'' returned non-zero exit status 134.

In [ ]:
import typing
T = typing.TypeVar('T')


In [ ]:
from chap4lib import *
from typing import Callable, Any

SyntaxError: invalid syntax (chap4lib.py, line 7)

## A1.

### Background

In our lectures (refer to `chap4.pdf`), we introduced the language $Lang_3$, which is a kernel imperative language. We discussed its syntax for expressions ($Exp_3$), definitions ($Def_3$), and commands ($Lang_3$). We also explored how to represent these constructs using Python classes and algebraic interfaces, as provided in `chap4lib.py`.

The key components are:

  * `Exp3Alg[ExpT]`: An algebra for building expressions. Its methods return objects of type `ExpT`.
  * `Def3Alg[ExpT, CmdT, DefT]`: An algebra for building definitions. Its methods (like `let` and `proc`) take expressions (of type `ExpT`) and commands (of type `CmdT`) and return objects of type `DefT`.
  * `Lang3Alg[ExpT, DefT, CmdT]`: An algebra for building commands. Its methods (like `assign`, `skip`, `seq`, `cond`, `loop`, `call_...`, `block`) take expressions (`ExpT`) and definitions (`DefT`) and return objects of type `CmdT`.

### Your Task

You are to implement Python functions with the given signatures. These functions will represent the given $Lang_3$ programs that define and calls a factorial procedure.




### A1-1.

**The $Lang_3$ Program to Implement:**

The program consists of an outer block that defines a procedure `factorial` and then calls it. The procedure itself uses an inner block and a `while` loop to compute the factorial.

```text
begin
  proc factorial(n) {             // procedure
    begin                         // block
      let y = ↑(1);               // definition
      while (not(↓n == 1)) do {   // command
        y := ↓y * ↓n;
        n := ↓n - 1
      }
    end
  }
  call factorial(↑3)              // Use lang.call_const() for call by value, constant
end
```

In [ ]:
def call_fact_pgm_const[ExpT, DefT, CmdT](
        exp: Exp3Alg[ExpT],
        de: Def3Alg[ExpT, CmdT, DefT],
        lang: Lang3Alg[ExpT, DefT, CmdT]
    ) -> CmdT:
    factorial_def = de.proc(
        "factorial", "n",
        lang.block(
        [de.let("y", exp.alloc(exp.cint(1)))],  # y=1を局所的に束縛する
        lang.loop(  # 第2引数: コマンドを単独で渡す
            exp.not_(exp.eq(exp.deref(exp.var("n")), exp.cint(1))),
            lang.seq(
                lang.assign("y", exp.mult(exp.deref(exp.var("y")), exp.deref(exp.var("n")))),
                lang.assign("n", exp.minus(exp.deref(exp.var("n")), exp.cint(1)))
                )
            )
        )
    )

    return lang.block(
        [factorial_def],
        lang.call_const("factorial", exp.alloc(exp.cint(3)))  # Correct format
    )

SyntaxError: expected '(' (<ipython-input-1-b57b12171e25>, line 1)

### A1-2.

**The $Lang_3$ Program to Implement:**

The program consists of an outer block that defines a procedure `factorial` and then calls it. The procedure parameter `n` is handled using call-by-value, mutable semantics. This means `n` behaves as a local, mutable variable within `factorial`, initialized with the value passed in the call.

```text
begin
  proc factorial(n) {             // procedure
    begin                         // block
      let y = ↑(1);               // definition
      while (not(↓n == 1)) do {   // command
        y := ↓y * ↓n;
        n := ↓n - 1
      }
    end
  }
  call factorial(3)               // Use lang.call_mutable() for call by value, mutable
end
```

In [ ]:
def call_fact_pgm_mutable[ExpT, DefT, CmdT](
    exp: Exp3Alg[ExpT],
    de: Def3Alg[ExpT, CmdT, DefT],
    lang: Lang3Alg[ExpT, DefT, CmdT]) -> CmdT:

    factorial_def = de.proc(
        "factorial", "n",
        lang.block(
            [de.let("y", exp.alloc(exp.cint(1)))],  # **修正なし**
            lang.loop(
                exp.not_(exp.eq(exp.deref(exp.var("n")), exp.cint(1))),  # **修正: `alloc()` を削除**
                lang.seq(
                    lang.assign("y", exp.mult(exp.deref(exp.var("y")), exp.deref(exp.var("n")))),
                    lang.assign("n", exp.minus(exp.deref(exp.var("n")), exp.cint(1)))
                )
            )
        )
    )

    return lang.block(
        [factorial_def],
        lang.call_mutable("factorial", exp.cint(3))  # **修正: `alloc()` を削除**
    )


### A1-3.

**The $Lang_3$ Program to Implement:**

The program consists of an outer block that defines the `factorial` procedure and a variable `x`. It then calls `factorial` passing `x` by reference.

```text
begin
  proc factorial(n) {             // procedure
    begin                         // block
      let y = ↑(1);               // definition
      while (not(↓n == 1)) do {   // command
        y := ↓y * ↓n;
        n := ↓n - 1
      }
    end
  }
  let x = ↑3;
  call p(x)                       // Use lang.call_ref() for call by reference
end
```

In [ ]:
def call_fact_pgm_ref[ExpT, DefT, CmdT](
    exp: Exp3Alg[ExpT],
    de: Def3Alg[ExpT, CmdT, DefT],
    lang: Lang3Alg[ExpT, DefT, CmdT]) -> CmdT:
    factorial_def = de.proc(
        "factorial", "n",
        lang.block(
            [de.let("y",exp.alloc(exp.cint(1)))],
            lang.loop(
                exp.not_(exp.eq(exp.deref(exp.var("n")),exp.cint(1))),
                lang.seq(
                    lang.assign("y",exp.mult(exp.deref(exp.var("y")),exp.deref(exp.var("n")))),
                    lang.assign("n",exp.minus(exp.deref(exp.var("n")),exp.cint(1))),
                )
            )
        )
      )

    return lang.block(
        [factorial_def,de.let("x",exp.alloc(exp.cint(3)))],
        lang.call_ref("factorial","x")
    )

## A2.

This assignment requires you to implement key components of an interpreter for $Lang_3$, an imperative language with explicit memory management, definitions, and procedures. You will be working with an algebraic approach, implementing Python classes that define the semantics for expressions ($Exp_3$), definitions ($Def_3$), and the language commands ($Lang_3$) themselves.

The primary objectives are:
* To understand and implement the evaluation semantics for expressions, including those with side effects (memory allocation).
* To implement the semantics for definitions, including procedure definitions with static (lexical) scoping.
* To implement the big-step operational semantics for language commands, including assignments, control flow, blocks, and procedure calls (call-by-value constant, call-by-value mutable, and call-by-reference) using static scope.
* To gain practical experience with functional approaches to interpreter writing in Python, using higher-order functions to represent computations.

You **must** use the interfaces and data structures provided in `chap4lib.py`.

### Core Types for Evaluation Functions

Your implementations will revolve around functions that take the current state `(EnvType, MemType)` and return new states or values. The key evaluation function types (as seen in the slides [cite: 34, 46, 81]) are:

* `EvalResultType = Callable[[StateType], tuple[Value, MemType]]`: For evaluating expressions in $Exp_3$. It takes a state and returns the resulting `Value` and the (potentially modified) `MemType`.
* `EvalDefResultType = Callable[[StateType], StateType]`: For evaluating definitions in $Def_3$. It takes a state and returns a new state `(EnvType, MemType)` reflecting the new bindings.
* `EvalCmdResultType = Callable[[StateType], MemType]`: For executing commands in $Lang_3$. It takes a state `(EnvType, MemType)` and returns the final `MemType` after the command's execution.

### Tasks

You need to define and implement the following three Python classes:


#### Task 1: Implement `Exp3Eval`

This class implements the `Exp3Alg` interface and defines the evaluation semantics for $Exp_3$ expressions.

In [ ]:
EvalResultType = Callable[[StateType], tuple[Value, MemType]]
AddrType = int

def apply_binary_op[T] (e1: EvalResultType, e2: EvalResultType, typ: type | None, f: Callable[[T, T], Value]) -> EvalResultType:
    def _eval(s: StateType) -> tuple [Value, MemType]:
        v1, m1 = e1(s)
        v2, m2 = e2((s[0], m1))

        if typ and not (isinstance(v1, typ) and isinstance(v2, typ)):
            raise ValueError(f"Type error: {v1} and {v2} must be {typ}.")
        elif not isinstance(v1, type(v2)):
            raise ValueError(f"Type error: {v1} and {v2} must be the same type")

        v1_val, v2_val = v1.v, v2.v
        return (f(v1_val, v2_val), m2)
    return _eval
class Exp3Eval(Exp3Alg[EvalResultType]):
  def cint(self, val: int) -> EvalResultType:
    return lambda s: (VInt(val), s[1])
  def cbool(self, val: bool) -> EvalResultType:
    return lambda s: (VBool(val), s[1])
  def addr(self, val_addr: AddrType) -> EvalResultType:
    return lambda s: (VAddr(val_addr), s[1])

  def var(self,symb:str) -> EvalResultType:
    def _eval(s: StateType) -> tuple[Value,MemType]:
      env,mem = s
      val_in_env = value_of(env,symb)
      if val_in_env is None:
        raise ValueError(f"Runtime error:Variable '{symb}' not defined")
      return val_in_env,mem
    return _eval

  def not_(self,e:EvalResultType) -> EvalResultType:
    def _eval(s1:StateType) -> tuple[Value,MemType]:
      v,m2 = e(s1)
      if isinstance(v,VBool):
        return (VBool(not v.v), m2)
      else:
        raise ValueError("Type error: 'not' expects a boolean value")
    return _eval
  #二項演算

  def plus(self,e1:EvalResultType,e2:EvalResultType) -> EvalResultType:
    def _plus(v1:int,v2:int) -> Value: return VInt(v1 + v2)
    return  apply_binary_op(e1,e2,VInt,_plus)

  def minus(self,e1:EvalResultType,e2:EvalResultType) -> EvalResultType:
    def _minus(v1:int,v2:int) -> Value: return VInt(v1 - v2)
    return apply_binary_op(e1,e2,VInt,_minus)

  def mult(self,e1:EvalResultType,e2:EvalResultType) -> EvalResultType:
    def _mult(v1:int,v2:int) -> Value: return VInt(v1 * v2)
    return apply_binary_op(e1,e2,VInt,_mult)

  def div(self,e1:EvalResultType,e2:EvalResultType) -> EvalResultType:
    def _div(v1:int,v2:int) -> Value:
      if v2 != 0:
        return VInt(v1 // v2)
      else:
        raise ValueError("Runtime error: Division by zero")
    return apply_binary_op(e1,e2,VInt,_div)

  def and_(self,e1:EvalResultType,e2:EvalResultType) -> EvalResultType:
    def _and(v1:bool,v2:bool) -> Value:
      return VBool(v1 and v2)
    return apply_binary_op(e1,e2,VBool,_and)

  def or_(self,e1:EvalResultType,e2:EvalResultType) -> EvalResultType:
    def _or(v1:bool,v2:bool) -> Value:
      return VBool(v1 or v2)
    return apply_binary_op(e1,e2,VBool,_or)

  def eq(self,e1:EvalResultType,e2:EvalResultType) -> EvalResultType:
    def _eq(v1:Value,v2:Value) -> Value:
      return VBool(v1 == v2)
    return apply_binary_op(e1, e2, None, _eq)

  def leq(self,e1:EvalResultType,e2:EvalResultType) -> EvalResultType:
    def _leq(v1:int,v2:int) -> Value:
      return VBool(v1 <= v2)
    return apply_binary_op(e1,e2,VInt,_leq)

  def alloc(self,e:EvalResultType) -> EvalResultType:
    def _eval(s:StateType) -> tuple[Value,MemType]:
      v,mem1 = e(s)
      new_addr,mem2 = add_mem(mem1,v)
      return VAddr(new_addr),mem2
    return _eval

  def deref(self,e:EvalResultType) -> EvalResultType:
    def _eval(s:StateType) -> tuple[Value,MemType]:
      addr,mem = e(s)
      if not isinstance(addr,VAddr):
        raise ValueError("Runtime error: Dereference operator expects an address.")
      stored_val = get_mem(mem,addr.v)
      if stored_val is None:
        raise ValueError(f"Runtime error: Address {addr.v} not found")
      return stored_val,mem
    return _eval

#### Task 2: Implement `Def3Eval`

This class implements the `Def3Alg` interface and defines the semantics for $Def_3$ definitions.

**Hint for `proc` closure:** A procedure value (closure for static scope) should store `(parameter_name, body_command, definition_environment)`. You can define a simple class `VProc(Value)` to encapsulate this tuple or store the tuple directly in the environment. Ensure that this value can be retrieved and correctly interpreted during procedure calls.

In [ ]:
EvalDefResultType = Callable[[StateType], StateType]
EvalCmdResultType = Callable[[StateType], MemType]

class VProc(Value):
    def __init__(self, p: str, x: str, c: EvalCmdResultType, e: EnvType):
        self.p = p  # 手続き名
        self.x = x  # パラメータ名
        self.c = c  # 手続きの本体（コマンド評価）
        self.e = e  # 定義時の環境（静的スコープのため）


class Def3Eval(Def3Alg[EvalResultType, EvalDefResultType, EvalCmdResultType]):
  def let(self,x:str,e:EvalResultType) -> EvalDefResultType:
    def _eval(s:StateType) -> StateType:
      v,mem2 = e(s)
      env2 = extend_env(s[0],x,v)
      return env2,mem2
    return _eval

  def proc(self,p:str,x:str,c:EvalCmdResultType) -> EvalDefResultType:
    def _eval(s:StateType) -> StateType:
      env2 = extend_env(s[0],p,VProc(p,x,c,s[0])) # (手続き名,クロージャ)を用いて適応時の環境を拡張
      return env2,s[1] #環境とメモリを渡す
    return _eval

#### Task 3: Implement `Lang3EvalSS` (Static Scoping)

This class implements the `Lang3Alg` interface and defines the big-step operational semantics for $Lang_3$ commands, incorporating **static scoping** for procedures.


In [ ]:

class Lang3EvalSS(Lang3Alg[EvalResultType,EvalDefResultType,EvalCmdResultType]):
  def assign(self,x:str,e:EvalResultType) -> EvalCmdResultType:
    def _eval(s:StateType) -> MemType:
      env,mem1 = s

      # 環境から変数xに束縛されている値を参照
      addr_of_x = value_of(env,x)
      if not isinstance(addr_of_x,VAddr):
        raise ValueError(f"variable '{x}' is not a valid reference")
      # sをeで評価する
      v,mem2 = e(s)

      final_mem = update_mem(mem2,addr_of_x.v,v)
      return final_mem
    return _eval

  def skip(self) -> EvalCmdResultType:
    def _eval(s:StateType) -> MemType:
      return s[1] # そのまま現在のメモリを渡す
    return _eval

  def seq(self,c1:EvalCmdResultType,c2:EvalCmdResultType) -> EvalCmdResultType:
    def _eval(s:StateType) -> MemType:
      env,_ = s
      m1 = c1(s) # コマンドでstateを評価するとmemになることに注意
      m2 = c2((env,m1))
      return m2
    return _eval

  def cond(self,
           e:EvalResultType,
           c1:EvalCmdResultType,
           c2:EvalCmdResultType) -> EvalCmdResultType:
    def _eval(s1:StateType) -> MemType:
      env,mem1 = s1
      v,mem2 = e(s1)

      # vはVBool型であるのを想定
      if not isinstance(v,VBool):
        raise ValueError("Condition in 'if' statement must be boolean")
      # 元の環境のもと更新されたメモリを用いてs2を作成
      s2 = (env,mem2)
      # vの値がTrue/Falseで評価を変える -> 元の環境を持っておく必要あり
      if v.v:
        return c1(s2)
      else:
        return c2(s2)
    return _eval

  def loop(self,e:EvalResultType,c_body:EvalCmdResultType) -> EvalCmdResultType:
    def _eval(s1:StateType) -> MemType:
      env,mem1 = s1
      val_e, mem2 = e(s1)
      if not isinstance(val_e,VBool):
        raise ValueError("Condition in 'while' statement must be boolean")
      if val_e.v:
        s2 = (env,mem2)
        s3 = (env,c_body(s2))
        return _eval(s3)
      else:
        return mem2
    return _eval

  def call_const(self,p:str,e:EvalResultType) -> EvalCmdResultType:
    def _eval(s:StateType) -> MemType:
      env,mem1 = s

      proc_def = value_of(env,p)
      if not isinstance(proc_def,VProc):
        raise ValueError(f"Procedure '{p}' is not a valid procedure.")

      param_name,c_body,def_env = proc_def.x,proc_def.c,proc_def.e

      v,mem2 = e(s)

      env_p = extend_env(def_env,param_name,v)

      env_p = extend_env(env_p,p,proc_def)

      return c_body((env_p,mem2))
    return _eval

  # call_murtableは引数はstrじゃなくeはEvalResultType
  def call_mutable(self, p: str, e: EvalResultType) -> EvalCmdResultType:
    def _eval(s: StateType) -> MemType:
        env, mem1 = s

        # `p` の定義を取得
        proc_def = value_of(env, p)
        if not isinstance(proc_def, VProc):
            raise ValueError(f"Procedure '{p}' is not a valid procedure.")

        param_name, body, def_env = proc_def.x, proc_def.c, proc_def.e

        # `e` を評価し、その値 `v` と更新された `mem2` を取得
        v, mem2 = e(s)

        # 新しいメモリアドレス `rx` を確保
        addr_x, mem3 = add_mem(mem2, v)  # メモリに `v` を保存し、アドレスを取得

        # 定義環境を拡張して `x` に `addr_x` をバインド
        env_p = extend_env(def_env, param_name, VAddr(addr_x))

        # 手続きを新しい環境で評価
        mem_final = body((env_p, mem3))
        return mem_final

    return _eval


  def call_ref(self,p:str,x:str) -> EvalCmdResultType:
    def _eval(s:StateType) -> MemType:
      env,mem1 = s

      # 手続きをもとに環境からx,command,env)を取得
      proc_def = value_of(env, p)
      if not isinstance(proc_def, VProc):
          raise ValueError(f"Procedure '{p}' is not a valid procedure.")

      addr_x = value_of(env,x)
      param_name,body,def_env = proc_def.x,proc_def.c,proc_def.e

      # 手続きのx_{p}と参照rのtupleペアを環境を拡張
      env_p = extend_env(def_env,param_name,addr_x)

      # (p,(x_{p},c,env)) => (p,proc_def)を環境を拡張
      env_p = extend_env(env_p,p,proc_def)

      mem_final = body((env_p,mem1))
      return mem_final
    return _eval

  def block(self, d: list[EvalDefResultType], c: EvalCmdResultType) -> EvalCmdResultType:
    def _eval(s: StateType) -> MemType:
        env1, mem1 = s  # 初期状態を取得

        # 定義リストの要素を順次適用
        for definition in d:
            env1, mem1 = definition((env1, mem1))

        # コードを評価しメモリ状態を更新
        mem2 = c((env1, mem1))

        return mem2

    return _eval

## A3.

This assignment builds upon your understanding of $Lang_3$ by requiring you to implement its **small-step operational semantics**. Unlike big-step semantics which define the overall result of a command, small-step semantics describe how commands execute one step at a time. This approach is crucial for understanding more complex language features like concurrency and for a finer-grained view of program execution.

You will define Python classes to represent $Lang_3$ commands. Each command class will encapsulate its own single-step execution logic via a `step` method. This approach uses polymorphism to avoid large conditional statements for different command types.

You will focus on the core commands: assignment, skip, sequence, conditional, and loop. **Procedure calls and blocks are excluded from this assignment** as their small-step semantics were not covered in detail in the lecture slides.

The primary objectives are:
* To understand and implement small-step operational semantics.
* To represent program syntax as data structures (command objects) with associated behavior (step methods).
* To implement state transition logic within each command class based on formal semantic rules.

### Core Concepts for Small-Step Semantics

* **Configuration:** A configuration is a pair `⟨c, Mem⟩`, where `c` is the command (or part of a command) remaining to be executed, and `Mem` is the current memory state.
* **Transition Relation (`↪_Env`):** Describes a single step of computation: `⟨c₁, Mem₁⟩ ↪_Env ⟨c₂, Mem₂⟩`.
* **Terminal Configuration:** Execution stops when a command `c` in state `(Env, Mem)` has no further steps. For this assignment, a command sequence terminates when it reduces to a single `skip` command in a configuration `⟨skip, Mem⟩`.

### 4. Tasks

#### Step 1: Command Representation (`SmallStepCmd`) with `step` Methods

First, you need to define Python classes to represent the $Lang_3$ commands. Each class will now also include a `step` method that defines how an instance of that command takes a single execution step.

Define the following classes. We'll use `SmallStepCmdType` as a type hint for these command objects. You should also define your `Exp3Eval` class from the previous assignment or ensure it's available, as its `EvalResultType` is needed.

```python
class SmallStepCmd: # base class
    def step(self,
             env: EnvType,
             current_mem: MemType,
             exp_eval_instance: Exp3Eval # Use forward reference or import
            ) -> Tuple[SmallStepCmd, MemType]:
        """Abstract method for performing one step of execution."""
        raise NotImplementedError
    # Consider adding __repr__ for debugging all command classes

class AssignCmd(SmallStepCmd):
    def __init__(self, var_name: str, expr_eval_func: 'EvalResultType'):
        self.var_name = var_name
        self.expr_eval_func = expr_eval_func

    def step(self, env: EnvType, current_mem: MemType, exp_eval_instance: Exp3Eval) -> Tuple[SmallStepCmd, MemType]:
        # Implements Rule S1 (see Part 3 for details)
        raise NotImplementedError # To be implemented by student

class SkipCmd(SmallStepCmd):
    def __init__(self):
        pass # No specific data for SkipCmd

    def step(self, env: EnvType, current_mem: MemType, exp_eval_instance: Exp3Eval) -> Tuple[SmallStepCmd, MemType]:
        # A SkipCmd is already in a "reduced" form.
        # It doesn't take a further step to a different command.
        # The interpreter loop (Part 4) will identify SkipCmd as a terminal state.
        return (self, current_mem)

class SeqCmd(SmallStepCmd):
    def __init__(self, cmd1: SmallStepCmd, cmd2: SmallStepCmd):
        self.cmd1 = cmd1
        self.cmd2 = cmd2

    def step(self, env: EnvType, current_mem: MemType, exp_eval_instance: Exp3Eval) -> Tuple[SmallStepCmd, MemType]:
        # Implements Rules S2 and S3 (see Part 3 for details)
        raise NotImplementedError # To be implemented by student

class CondCmd(SmallStepCmd):
    def __init__(self, cond_expr_eval_func: 'EvalResultType', then_cmd: SmallStepCmd, else_cmd: SmallStepCmd):
        self.cond_expr_eval_func = cond_expr_eval_func
        self.then_cmd = then_cmd
        self.else_cmd = else_cmd

    def step(self, env: EnvType, current_mem: MemType, exp_eval_instance: Exp3Eval) -> Tuple[SmallStepCmd, MemType]:
        # Implements Rules S4 and S5 (see Part 3 for details)
        raise NotImplementedError # To be implemented by student

class LoopCmd(SmallStepCmd):
    def __init__(self, cond_expr_eval_func: 'EvalResultType', body_cmd: SmallStepCmd):
        self.cond_expr_eval_func = cond_expr_eval_func
        self.body_cmd = body_cmd

    def step(self, env: EnvType, current_mem: MemType, exp_eval_instance: Exp3Eval) -> Tuple[SmallStepCmd, MemType]:
        # Implements Rules S6 and S7 (see Part 3 for details)
        raise NotImplementedError # To be implemented by student
```
*(Ensure `EvalResultType` and `Exp3Eval` are correctly defined and imported based on your project structure. `chap4lib.py` provides `EnvType`, `MemType`, `Value`. Your `Exp3Eval` solution from the previous assignment provides `EvalResultType`.)*

#### Step 2: Implementing `Lang3EvalSmall`

Next, implement the `Lang3EvalSmall` class. This class will implement the `Lang3Alg` interface. Its methods will **construct and return instances of your command representation classes** from Part 1.

* `Lang3EvalSmall` should implement `Lang3Alg[EvalResultType, typing.Any, SmallStepCmdType]`.
    * `ExpT` is `EvalResultType`.
    * `DefT` can be `typing.Any` as `block` (which uses definitions) is not part of this assignment's small-step implementation.
    * `T` (the return type of command construction methods) will be `SmallStepCmdType`.

**Implementation Details:**
* `assign(self, x: str, e: EvalResultType) -> SmallStepCmdType`: Returns an `AssignCmd(x, e)` instance.
* `skip(self) -> SmallStepCmdType`: Returns a `SkipCmd()` instance.
* `seq(self, c1: SmallStepCmdType, c2: SmallStepCmdType) -> SmallStepCmdType`: Returns a `SeqCmd(c1, c2)` instance.
* `cond(self, e: EvalResultType, c1: SmallStepCmdType, c2: SmallStepCmdType) -> SmallStepCmdType`: Returns a `CondCmd(e, c1, c2)` instance.
* `loop(self, e: EvalResultType, c: SmallStepCmdType) -> SmallStepCmdType`: Returns a `LoopCmd(e, c)` instance.
* **Excluded Methods:** The methods `block`, `call_const`, `call_mutable`, and `call_ref` **should raise `NotImplementedError`**.

#### Step 3: Implementing `step` Methods in Command Classes

This is the core of the small-step interpreter. Each command class you defined in Part 1 (`AssignCmd`, `SeqCmd`, `CondCmd`, `LoopCmd`) must implement its own `step` method. The `SkipCmd.step` method is trivial as it represents a completed step.

**Method Signature (to be implemented in each relevant command class):**
```python
# Inside each of your command classes (AssignCmd, SeqCmd, CondCmd, LoopCmd)
def step(self,
         env: EnvType,
         current_mem: MemType,
         exp_eval_instance: Exp3Eval # Your working Exp3Eval instance
        ) -> Tuple[SmallStepCmd, MemType]: # Return type uses forward reference
    # ... implementation specific to this command type based on S1-S7 ...
```

In [ ]:

class SmallStepCmd:
  def step(self, env: EnvType, current_mem: MemType, exp_eval_instance: Exp3Eval) -> tuple["SmallStepCmd", MemType]:
      raise NotImplementedError("step() はサブクラスで実装する必要があります。")


class AssignCmd(SmallStepCmd):
    def __init__(self, var_name: str, expr_eval_func: EvalResultType):
        self.var_name = var_name
        self.expr_eval_func = expr_eval_func

    def step(self, env: EnvType, current_mem: MemType, exp_eval_instance: Exp3Eval) -> tuple[SmallStepCmd, MemType]:
        addr_of_x = value_of(env, self.var_name)
        if not isinstance(addr_of_x, VAddr):  # 修正: 正しい型チェック
            raise ValueError(f"Variable '{self.var_name}' is not a valid reference.")

        v, mem2 = self.expr_eval_func((env, current_mem))  # 修正: `(env, current_mem)` を渡す
        final_mem = update_mem(mem2, addr_of_x.v, v)  # 修正: `addr_of_x.v`

        return SkipCmd(), final_mem  # 修正: `SkipCmd()` を返す

class SkipCmd(SmallStepCmd):
    def step(self, env: EnvType, current_mem: MemType, exp_eval_instance: Exp3Eval) -> tuple[SmallStepCmd, MemType]:
        return self, current_mem  # ✅ `SkipCmd` は状態を変えずにそのまま返す

class SeqCmd(SmallStepCmd):
    def __init__(self, cmd1: SmallStepCmd, cmd2: SmallStepCmd):
        self.cmd1 = cmd1
        self.cmd2 = cmd2

    def step(self, env: EnvType, current_mem: MemType, exp_eval_instance: Exp3Eval) -> tuple[SmallStepCmd, MemType]:
        next_cmd, new_mem = self.cmd1.step(env, current_mem, exp_eval_instance)
        if isinstance(next_cmd, SkipCmd):
            return SeqCmd(next_cmd, self.cmd2), new_mem  # ✅ `SkipCmd` でも `SeqCmd` に包んで返す
        elif isinstance(next_cmd,SeqCmd):
            return SeqCmd(next_cmd,self.cmd2), new_mem  # `c1` がまだ進行中なら更新

class CondCmd(SmallStepCmd):
    def __init__(self, cond_expr_eval_func: EvalResultType, then_cmd: SmallStepCmd, else_cmd: SmallStepCmd):
        self.cond_expr_eval_func = cond_expr_eval_func
        self.then_cmd = then_cmd
        self.else_cmd = else_cmd

    def step(self, env: EnvType, current_mem: MemType, exp_eval_instance: Exp3Eval) -> tuple[SmallStepCmd, MemType]:
        v, mem2 = self.cond_expr_eval_func((env, current_mem))
        if not isinstance(v, VBool):  # 修正: `VBool` の型チェックを追加
            raise ValueError("Condition in 'if' statement must be boolean")

        return (self.then_cmd if v.v else self.else_cmd), mem2

class LoopCmd(SmallStepCmd):
    def __init__(self, cond_expr_eval_func: EvalResultType, body_cmd: SmallStepCmd):
        self.cond_expr_eval_func = cond_expr_eval_func
        self.body_cmd = body_cmd
    def step(self, env: EnvType, current_mem: MemType, exp_eval_instance: Exp3Eval) -> tuple[SmallStepCmd, MemType]:
        cond_val, new_mem = self.cond_expr_eval_func((env, current_mem))

        if not isinstance(cond_val, VBool):
            raise ValueError("Loop condition must evaluate to a boolean value.")

        if cond_val.v:
            return SeqCmd(self.body_cmd, self), new_mem  # `self` を再帰的に使用
        else:
            return SkipCmd(), new_mem



In [ ]:
SmallStepCmdType = SmallStepCmd

class Lang3EvalSmall(Lang3Alg[EvalResultType, typing.Any, SmallStepCmdType]):
    """Small-Step Operational Semantics に基づく Lang3 の評価器"""

    def assign(self, x: str, e: EvalResultType) -> SmallStepCmdType:
        """代入コマンドのインスタンスを返す"""
        return AssignCmd(x, e)

    def skip(self) -> SmallStepCmdType:
        """skip コマンドのインスタンスを返す"""
        return SkipCmd()

    def seq(self,c1: SmallStepCmdType, c2: SmallStepCmdType) -> SmallStepCmdType:
        """シーケンスコマンドのインスタンスを返す"""
        return SeqCmd(c1, c2)

    def cond(self, e: EvalResultType, c1: SmallStepCmdType, c2: SmallStepCmdType) -> SmallStepCmdType:
        """条件分岐コマンドのインスタンスを返す"""
        return CondCmd(e, c1, c2)

    def loop(self, e: EvalResultType, c: SmallStepCmdType) -> SmallStepCmdType:
        """ループコマンドのインスタンスを返す"""
        return LoopCmd(e, c)